In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/home/srikar/Downloads/work/Financial1.csv")

In [3]:
df.head(25)

,0,303567,3584,w,0.000000
0,1,55590,3072,w,0.000000
1,0,303574,3584,w,0.026214
2,1,240840,3072,w,0.026214
3,1,55596,3072,r,0.078643
4,0,303581,3584,w,0.117964
5,1,55596,3072,w,0.117964
6,0,303588,3584,w,0.530841
7,1,55596,3072,w,0.530841
8,0,303595,3584,w,0.550502
9,1,240840,3072,w,0.550502


In [4]:
df.columns = ['asu', 'block_address','size', 'opcode','time_stamp']

In [5]:
df.head(25)

,asu,block_address,size,opcode,time_stamp
0,1,55590,3072,w,0.000000
1,0,303574,3584,w,0.026214
2,1,240840,3072,w,0.026214
3,1,55596,3072,r,0.078643
4,0,303581,3584,w,0.117964
5,1,55596,3072,w,0.117964
6,0,303588,3584,w,0.530841
7,1,55596,3072,w,0.530841
8,0,303595,3584,w,0.550502
9,1,240840,3072,w,0.550502


In [6]:
df.shape

(5334986, 5)

In [8]:
import tensorflow as tf
import helpers

tf.reset_default_graph()
sess = tf.InteractiveSession()

In [9]:
tf.__version__

'1.4.1'

In [10]:
PAD = 0
EOS = 1

vocab_size = 10
input_embedding_size = 20

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units

In [12]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')

In [13]:
decoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_inputs')

In [14]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

In [15]:
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, decoder_inputs)

In [16]:
encoder_cell = tf.contrib.rnn.LSTMCell(encoder_hidden_units)

encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs_embedded,
    dtype=tf.float32, time_major=True,
)

del encoder_outputs

In [17]:
encoder_final_state

LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 20) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 20) dtype=float32>)

In [18]:
decoder_cell = tf.contrib.rnn.LSTMCell(decoder_hidden_units)

decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
    decoder_cell, decoder_inputs_embedded,

    initial_state=encoder_final_state,

    dtype=tf.float32, time_major=True, scope="plain_decoder",
)

In [19]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocab_size)

decoder_prediction = tf.argmax(decoder_logits, 2)

In [20]:
#rnn optimizer

decoder_logits

<tf.Tensor 'fully_connected/BiasAdd:0' shape=(?, ?, 10) dtype=float32>

In [21]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)

loss = tf.reduce_mean(stepwise_cross_entropy)
train_op = tf.train.AdamOptimizer().minimize(loss)

In [22]:
sess.run(tf.global_variables_initializer())

In [24]:
batch_size = 100

batches = helpers.random_sequences(length_from=3, length_to=8,
                                   vocab_lower=2, vocab_upper=10,
                                   batch_size=batch_size)

print('batch head:')
for seq in next(batches)[:10]:
    print(seq)

batch head:
[6, 4, 5]
[6, 3, 5, 6, 4, 6, 4]
[8, 9, 8, 3]
[3, 4, 6, 4, 2, 2, 9, 3]
[9, 3, 4]
[6, 7, 6]
[4, 9, 7, 4]
[2, 9, 5, 2, 7, 9, 6]
[6, 6, 8, 2, 6, 8, 3]
[7, 4, 2, 4, 7, 3, 3, 2]


In [25]:
def next_feed():
    batch = next(batches)
    encoder_inputs_, _ = helpers.batch(batch)
    decoder_targets_, _ = helpers.batch(
        [(sequence) + [EOS] for sequence in batch]
    )
    decoder_inputs_, _ = helpers.batch(
        [[EOS] + (sequence) for sequence in batch]
    )
    return {
        encoder_inputs: encoder_inputs_,
        decoder_inputs: decoder_inputs_,
        decoder_targets: decoder_targets_,
    }

In [26]:
loss_track = []

In [27]:
max_batches = 3001
batches_in_epoch = 1000

try:
    for batch in range(max_batches):
        fd = next_feed()
        _, l = sess.run([train_op, loss], fd)
        loss_track.append(l)

        if batch == 0 or batch % batches_in_epoch == 0:
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            predict_ = sess.run(decoder_prediction, fd)
            for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
                print('  sample {}:'.format(i + 1))
                print('    input     > {}'.format(inp))
                print('    predicted > {}'.format(pred))
                if i >= 2:
                    break
            print()
except KeyboardInterrupt:
    print('training interrupted')

batch 0
  minibatch loss: 2.3477020263671875
  sample 1:
    input     > [2 8 7 3 8 8 6 4]
    predicted > [3 7 3 3 0 5 5 3 7]
  sample 2:
    input     > [6 9 4 5 7 0 0 0]
    predicted > [2 2 3 7 7 3 3 3 3]
  sample 3:
    input     > [4 7 7 4 5 2 0 0]
    predicted > [2 7 1 1 1 4 4 1 1]

batch 1000
  minibatch loss: 0.35061517357826233
  sample 1:
    input     > [2 9 9 3 7 0 0 0]
    predicted > [9 9 9 3 7 1 0 0 0]
  sample 2:
    input     > [3 7 2 8 8 2 9 0]
    predicted > [3 7 2 8 8 2 9 1 0]
  sample 3:
    input     > [4 9 6 0 0 0 0 0]
    predicted > [4 9 6 1 0 0 0 0 0]

batch 2000
  minibatch loss: 0.16141687333583832
  sample 1:
    input     > [9 8 4 5 5 5 6 3]
    predicted > [9 8 5 5 5 6 6 3 1]
  sample 2:
    input     > [9 7 6 5 2 9 0 0]
    predicted > [9 7 6 5 2 9 1 0 0]
  sample 3:
    input     > [4 6 3 0 0 0 0 0]
    predicted > [4 6 3 1 0 0 0 0 0]

batch 3000
  minibatch loss: 0.12105262279510498
  sample 1:
    input     > [9 6 6 6 5 4 0 0]
    predicted > [6 6 